<a href="https://colab.research.google.com/github/kacper-dobrow/tensorflow-course/blob/main/08_intro_to_NLP_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (e.g.: sequences of text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get helper functions

In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-11-05 15:37:38--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-11-05 15:37:38 (105 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset
The dataset we'll be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not)

See the original source here:

https://www.kaggle.com/competitions/nlp-getting-started


In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-11-05 15:37:41--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.12.27, 2607:f8b0:4025:815::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-11-05 15:37:41 (112 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize the data

Reading and writing files in Python:
https://realpython.com/read-write-files-python/

BUT you can use Pandas

Load and preprocess data in tf: https://www.tensorflow.org/tutorials/load_data/csv?hl=pl

In [6]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class - balanced or not
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In case the data was more imbalanced:

https://www.tensorflow.org/tutorials/structured_data/imbalanced_data?hl=pl

In [10]:
# how many total smaples are there
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Visualize random training samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}","(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Can you save
Can you save my
Can you save my heavydirtysoul?

---

Target: 0 (not real disaster)
Text:
@ThatPersianGuy @YOUNGSAFE ?? Eden Hazard as Harden is spot on flopping is identical

---

Target: 1 (real disaster)
Text:
the mv should just be them strutting like they mean it while buildings are burning up in the bg and flames everywhere how cool would that be

---

Target: 1 (real disaster)
Text:
shit is hard to get over but sometimes the tragedy means it's over soulja..

---

Target: 1 (real disaster)
Text:
U.S. record hurricane drought. http://t.co/fE9hIVfMxq

---



### Split data into training and validation datasets



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of data for validation split
                                                                            random_state=42)

In [14]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [15]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

Tokenization, Embedding

Word2vec embedding:
http://jalammar.github.io/illustrated-word2vec/

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary/dictionary. If none, there's no cap on the size of the vocabulary. If set to an integer, every word that doesn't fit into the dictionary is tokenized as <OOV>
                                    standardize="lower_and_strip_punctuation", # turns all uppercase to lowercase and removes dots, spaces etc
                                    split="whitespace", # text is split into tokens with whitespace
                                    ngrams=None, # whether to arrange text into n-word groups
                                    output_mode="int", # assigns an int token to every word
                                    output_sequence_length=None, # converts every string into a sequence of the same length - the length of the longest sequence - and adds 0's at the end if words are absent
                                    pad_to_max_tokens=False)

In [22]:
len(train_sentences[0].split())

7

In [23]:
# Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [24]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in the vocabulary
max_length =15 # how many words in a tweet does the model see

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length) # the vectorizer truncates or pads the tweets to this length

In [25]:
# fit the text vectorizer (it should be called tokenizer) to the training set
text_vectorizer.adapt(train_sentences)

In [26]:
# Create a sample sentence and vectorize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [30]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:\n{text_vectorizer([random_sentence])}")

Original text:
@Calum5SOS I need to stop doing this to myself???? @s_casualty        

Vectorized version:
[[4041    8  162    5  240  487   19    5  736 8765    0    0    0    0
     0]]


In [31]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocabulary: {len(words_in_vocab)}")
print(f"5 most common words: \n{top_5_words}")

Number of words in vocabulary: 10000
5 most common words: 
['', '[UNK]', 'the', 'a', 'in']


### Creating an embedding using an embedding layer
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

Most cared for parameters in the embedding layer
* `input_dim` = the size of the vocabulary
* `output_dim` = the size of our output embedding vector
* `input_length` = length of sequences being passed into the embedding layer

In [32]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape to the size of our vocabulary
                             output_dim=128, # GPUs work good with multiples of 8
                             input_length=max_length) # how long is each input

In [34]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded version:\n{embedding(text_vectorizer(random_sentence))}")

Original text:
THANKS!!!!! @COUNT DANTE.  :)  DO JOIN US BY FOLLOWING THE @ANNIHILATION ZONE.  JOHNNY.        

Embedded version:
[[-0.02361623 -0.0368399   0.0382773  ... -0.03674291  0.04634681
  -0.04304867]
 [-0.01489543 -0.02291628 -0.00895289 ... -0.04007447  0.00317595
   0.04555093]
 [-0.04130166  0.00561989 -0.04957107 ... -0.03503815 -0.0154383
  -0.01036691]
 ...
 [-0.01435757 -0.03190291  0.03656803 ... -0.00058116 -0.04784826
   0.03838892]
 [-0.01435757 -0.03190291  0.03656803 ... -0.00058116 -0.04784826
   0.03838892]
 [-0.01435757 -0.03190291  0.03656803 ... -0.00058116 -0.04784826
   0.03838892]]


In [36]:
# Check out a single token's embedding
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.02361623, -0.0368399 ,  0.0382773 , -0.00607731,  0.04017838,
       -0.00500299,  0.03188281,  0.00321501, -0.01241738, -0.04528409,
       -0.03252611,  0.02280439, -0.0239867 ,  0.00082789,  0.02925562,
        0.04987926,  0.03994201, -0.04952023, -0.02435465,  0.01162374,
       -0.027941  ,  0.01997521,  0.01492937,  0.00825934, -0.04564429,
        0.02473552,  0.04927311,  0.03671643, -0.04906862, -0.03153998,
        0.03619302, -0.00309885, -0.01934789,  0.02451375, -0.02491096,
       -0.0192274 , -0.01330134,  0.01662531,  0.02355452, -0.0094952 ,
        0.03604325,  0.0407383 ,  0.02029655,  0.03175432,  0.04651363,
        0.0487748 ,  0.03095851, -0.01344027,  0.04540796,  0.04514186,
        0.0206827 ,  0.02874103,  0.0274038 , -0.00433029,  0.04174164,
       -0.00957081,  0.03907411, -0.01419673,  0.0405135 ,  0.01215183,
        0.0061826 ,  0.02565671, -0.03428028, -0.01812883,  0.03563023,
       -0.039241

In [38]:
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02361623, -0.0368399 ,  0.0382773 , -0.00607731,  0.04017838,
        -0.00500299,  0.03188281,  0.00321501, -0.01241738, -0.04528409,
        -0.03252611,  0.02280439, -0.0239867 ,  0.00082789,  0.02925562,
         0.04987926,  0.03994201, -0.04952023, -0.02435465,  0.01162374,
        -0.027941  ,  0.01997521,  0.01492937,  0.00825934, -0.04564429,
         0.02473552,  0.04927311,  0.03671643, -0.04906862, -0.03153998,
         0.03619302, -0.00309885, -0.01934789,  0.02451375, -0.02491096,
        -0.0192274 , -0.01330134,  0.01662531,  0.02355452, -0.0094952 ,
         0.03604325,  0.0407383 ,  0.02029655,  0.03175432,  0.04651363,
         0.0487748 ,  0.03095851, -0.01344027,  0.04540796,  0.04514186,
         0.0206827 ,  0.02874103,  0.0274038 , -0.00433029,  0.04174164,
        -0.00957081,  0.03907411, -0.01419673,  0.0405135 ,  0.01215183,
         0.0061826 ,  0.02565671, -0.03428028, -0.01812883,  0.03563023,
  

## Modelling a text dataset

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

* Model 0: Naive Bayes
* Model 1: Feed-forward NN
* Model 2: LSTM
* Model 3: GRU
* Model 4: Bidirectional-LSTM
* Model 5: 1D CNN
* Model 6: TFHub pretrained feature extractor
* Model 7: Same as 6 but with 10% data

### Model 0: Baseline

Create a baseline as a benchmark for our future experiments

We'll use SKLearn's Multinomial Naive Bayes using the TF-IDF formula to convert words to numbers

First we use non-deep-learning model as a reference because of its speed. Next we use deep learning models to improve the performance.

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization pipeline (equivalent to keras.Sequential)
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [41]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels) # Equivalent to keras .evaluate method
print(f"Baseline model achieves accuracy of: {baseline_score*100:.2f}%")

Baseline model achieves accuracy of: 79.27%


In [42]:
# Make predictions
model_0_preds = model_0.predict(val_sentences) # just like in keras

### Create a function for comparing performance

We evaluate
* Accuracy
* Precision
* Recall
* F1-score

https://scikit-learn.org/stable/modules/model_evaluation.html


In [56]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1-score of a binary classification
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)
  # Calculate model precision, recall and f1-score using "weighted" average to account for label imbalance
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [57]:
# get baseline results
model_0_results = calculate_results(val_labels, model_0_preds)
model_0_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed-forward dense model

In [58]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [64]:
# Build a functional API
import tensorflow as tf
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # Inputs are 1D strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the vectorized inputs
x = layers.GlobalAveragePooling1D()(x) # Reduce dimensionality
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs, name="model_1_dense")

In [65]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [66]:
# Compile the model
model_1.compile(optimizer=tf.optimizers.Adam(),
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [67]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20231105-193253
Epoch 1/5
215/215 [==============================] - 6s 22ms/step - loss: 0.6121 - accuracy: 0.6898 - val_loss: 0.5336 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4417 - accuracy: 0.8213 - val_loss: 0.4689 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3467 - accuracy: 0.8616 - val_loss: 0.4563 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.2846 - accuracy: 0.8911 - val_loss: 0.4650 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.2372 - accuracy: 0.9107 - val_loss: 0.4804 - val_accuracy: 0.7874


In [68]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4804 - accuracy: 0.7874


[0.4804351031780243, 0.787401556968689]

In [70]:
# Make some predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 7ms/step


(762, 1)

In [72]:
model_1_pred_probs[:10]

array([[0.3474779 ],
       [0.850074  ],
       [0.9979714 ],
       [0.1286889 ],
       [0.11433835],
       [0.94312257],
       [0.9128659 ],
       [0.9938656 ],
       [0.9690019 ],
       [0.25055087]], dtype=float32)

In [74]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [75]:
# Calculate model_1 results
model_1_results = calculate_results(val_labels,model_1_preds)
model_1_results

{'accuracy': 0.7874015748031497,
 'precision': 0.7923216200196284,
 'recall': 0.7874015748031497,
 'f1': 0.7844114973523961}

In [76]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(model_0_results.values()))

array([False, False, False, False])

Baseline is doing better

## Visualize learned embeddings

In [77]:
# Get the vocabulary from the text tokenization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [79]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [84]:
# Get the weight matirx of embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

Let's see how we can visualize it

https://projector.tensorflow.org/

https://www.tensorflow.org/text/guide/word_embeddings?hl=pl

In [87]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [88]:
# Download foles from colab to import to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural networks (RNNs)

RNNs are useful for sequence data

The premise of a recurrent neural network is to use the representation of a previous input to aid the  representation of the later input.

https://www.youtube.com/watch?v=qjrad0V0uJE

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2: LSTM

LSTM - long short term memory

https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

In [94]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64, return_sequences=True)(x) # when stacking recurrent layers you need to set return_sequences to True. Otherwise the layer reduces dimensions
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [95]:
# Get s aummary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [96]:
# Compile
model_2.compile(optimizer="Adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [97]:
# Fit the model
history_model_2 = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20231105-203329
Epoch 1/5
215/215 [==============================] - 12s 40ms/step - loss: 0.2166 - accuracy: 0.9259 - val_loss: 0.5461 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 7s 34ms/step - loss: 0.1577 - accuracy: 0.9432 - val_loss: 0.5366 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 10s 48ms/step - loss: 0.1269 - accuracy: 0.9527 - val_loss: 0.7438 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 11s 52ms/step - loss: 0.1045 - accuracy: 0.9604 - val_loss: 0.8319 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 12s 54ms/step - loss: 0.0873 - accuracy: 0.9663 - val_loss: 0.9287 - val_accuracy: 0.7730


In [99]:
# Make predictions with model 2
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 6ms/step


array([[8.2182279e-03],
       [9.0679294e-01],
       [9.9986178e-01],
       [5.4437485e-02],
       [5.9581868e-04],
       [9.9934709e-01],
       [7.8873384e-01],
       [9.9991089e-01],
       [9.9984539e-01],
       [3.1563264e-01]], dtype=float32)

In [102]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [104]:
# Calculate model 2 results
model_2_results = calculate_results(val_labels,
                                    model_2_preds)
model_2_results

{'accuracy': 0.7729658792650919,
 'precision': 0.7747736761679446,
 'recall': 0.7729658792650919,
 'f1': 0.7708425630155245}

### Model 3: GRU

Gated recurrent unit (GRU) has similar features to LSTM but less trainable parameters

https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

In [112]:
# Create the same model but with a GRU cell
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x) # if wished to stack recurrent layers, set return_sequences=True
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [114]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [113]:
# Compile the model
model_3.compile(optimizer="Adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [115]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20231105-205349
Epoch 1/5
215/215 [==============================] - 14s 41ms/step - loss: 0.1532 - accuracy: 0.9384 - val_loss: 0.8330 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 10s 46ms/step - loss: 0.0812 - accuracy: 0.9680 - val_loss: 0.9105 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 11s 50ms/step - loss: 0.0712 - accuracy: 0.9711 - val_loss: 0.9969 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 10s 46ms/step - loss: 0.0544 - accuracy: 0.9752 - val_loss: 1.1643 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 11s 53ms/step - loss: 0.0520 - accuracy: 0.9766 - val_loss: 1.3382 - val_accuracy: 0.7822


In [116]:
# Make predictions
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 2s 11ms/step


array([[1.5868101e-04],
       [7.3154861e-01],
       [9.9988639e-01],
       [5.2936822e-02],
       [5.1658324e-05],
       [9.9965054e-01],
       [9.1521353e-01],
       [9.9994606e-01],
       [9.9988276e-01],
       [6.4849794e-01]], dtype=float32)

In [117]:
# Convert pred probs to preds
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [118]:
# Calculate model 3 results
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 0.7821522309711286,
 'precision': 0.7860403042839984,
 'recall': 0.7821522309711286,
 'f1': 0.779380517102918}

### Model 4: Bidirectional RNN

Bidirectional RNN feeds the representation of the former as well as the latter element in the sentence to the representation of the current element

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional

In [125]:
# Create the model
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs)

In [126]:
# Get the summary
model_4.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable params: 1378945 (

In [127]:
# Compile model
model_4.compile(optimizer="Adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])

In [128]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20231105-211223
Epoch 1/5
215/215 [==============================] - 17s 49ms/step - loss: 0.1084 - accuracy: 0.9666 - val_loss: 1.0184 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0522 - accuracy: 0.9775 - val_loss: 1.1232 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0479 - accuracy: 0.9778 - val_loss: 1.0811 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0427 - accuracy: 0.9810 - val_loss: 1.3073 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0377 - accuracy: 0.9818 - val_loss: 1.6485 - val_accuracy: 0.7651


In [129]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 10ms/step


array([[3.9936062e-05],
       [4.7330683e-01],
       [9.9997520e-01],
       [4.1201659e-02],
       [5.8274172e-06],
       [9.9971759e-01],
       [9.3395048e-01],
       [9.9998808e-01],
       [9.9996871e-01],
       [9.9958915e-01]], dtype=float32)

In [132]:
# Convert to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [133]:
# Calculate the results
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 0.7650918635170604,
 'precision': 0.7675147482403992,
 'recall': 0.7650918635170604,
 'f1': 0.7624764882687336}

https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21